# Brief sketch of Julia's secret sauce

Julia's *secret sauce*

- **Ahead-of-time compilation**
- **Multiple dispatch**
- **Abstract types**

The following is not needed if running from the REPL:

In [1]:
using Pkg
Pkg.activate(temp=true)
Pkg.add("InteractiveUtils")
using InteractiveUtils

  Activating new environment at `/var/folders/4n/gvbmlhdc8xj973001s6vdyw00000gq/T/jl_VX4P55/Project.toml`
   Resolving package versions...
    Updating `/private/var/folders/4n/gvbmlhdc8xj973001s6vdyw00000gq/T/jl_VX4P55/Project.toml`
  [b77e0a4c] + InteractiveUtils
    Updating `/private/var/folders/4n/gvbmlhdc8xj973001s6vdyw00000gq/T/jl_VX4P55/Manifest.toml`
  [2a0f44e3] + Base64
  [b77e0a4c] + InteractiveUtils
  [d6f4376e] + Markdown


In [2]:
# Just-in-time compilation

**Note.** Because of some more recent fancy optimizations, this demo only
works on Julia versions 1.3 and older.

In [3]:
2 + 2

4

In [4]:
typeof(42.0)

Float64

## Just-in-time compilation

In [5]:
add(x, y) = x + y

@elapsed add(3, 5)

7.3e-8

In [6]:
@elapsed add(2, 4)

7.9e-8

In [7]:
@elapsed add(1 + 2im, 4 + 3im)

1.18e-7

In [8]:
@elapsed add(3 + 6im, 7 - 5im)

9.0e-8

## Multiple dispatch

Objects don't own methods, functions do:

In [9]:
y = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [10]:
typeof(y)

Matrix{Int64} (alias for Array{Int64, 2})

Uncomment to see error thrown:

add(4, y)

In [11]:
add(x::Int, y::Array{Int,2}) = x .+ y

add (generic function with 2 methods)

In [12]:
add(4, y)

2×2 Matrix{Int64}:
 5  6
 7  8

In [13]:
methods(add)

# 2 methods for generic function "add":
[1] add(x::Int64, y::Matrix{Int64}) in Main.##266 at string:1
[2] add(x, y) in Main.##266 at string:1

Uncomment to see error thrown:

add(4.0, y)

## Abstract types

In [14]:
supertype(Int)

Signed

In [15]:
supertype(Signed)

Integer

In [16]:
supertype(Integer)

Real

In [17]:
subtypes(Real)

7-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 FixedPointNumbers.FixedPoint
 Integer
 Rational
 StatsBase.PValue
 StatsBase.TestStat

In [18]:
4 isa Real

true

In [19]:
Bool <: Integer

true

In [20]:
String <: Integer

false

In [21]:
add(x::Real, y::Matrix) = x .+ y

add (generic function with 3 methods)

In [22]:
add(4.0, rand(Bool, 2, 3))

2×3 Matrix{Float64}:
 4.0  4.0  5.0
 5.0  5.0  5.0

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*